# Acquisition Functions & the Full BO Loop

**Bayesian Optimisation Series · Notebook 3 of 3**

This notebook:
1. Implements EI, PI, and UCB acquisition functions from scratch
2. Visualises all three on the same GP posterior — showing how they pick different next points
3. Runs a complete Bayesian Optimisation loop on a multi-modal test function
4. Compares BO against random search
5. Demonstrates the exploration–exploitation trade-off via the UCB β parameter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib import rcParams

rcParams['figure.figsize'] = (12, 5)
rcParams['font.size'] = 12
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

np.random.seed(42)

## 1. GP Infrastructure (from Notebook 1)

In [ ]:
def rbf_kernel(X1, X2, l=1.0, sf=1.0):
    sq = np.sum(X1**2, 1).reshape(-1, 1) + np.sum(X2**2, 1).reshape(1, -1) - 2 * X1 @ X2.T
    return sf**2 * np.exp(-0.5 * sq / l**2)


def gp_posterior(X_train, y_train, X_test, l=1.0, sf=1.0, noise=1e-6):
    K = rbf_kernel(X_train, X_train, l, sf) + noise * np.eye(len(X_train))
    K_s = rbf_kernel(X_train, X_test, l, sf)
    K_ss = rbf_kernel(X_test, X_test, l, sf)

    L = np.linalg.cholesky(K)
    alpha = np.linalg.solve(L.T, np.linalg.solve(L, y_train))
    mu = (K_s.T @ alpha).ravel()

    v = np.linalg.solve(L, K_s)
    var = np.diag(K_ss) - np.sum(v**2, axis=0)
    var = np.maximum(var, 1e-10)

    return mu, var

## 2. The Three Canonical Acquisition Functions

All three are derived from the GP posterior $\mathcal{N}(\mu_n(x), \sigma_n^2(x))$ and the current best $f^* = \max\{y_1, \ldots, y_n\}$.

**PI:** $\alpha_{PI}(x) = \Phi\left(\frac{\mu_n(x) - f^* - \xi}{\sigma_n(x)}\right)$

**EI:** $\alpha_{EI}(x) = (\mu_n(x) - f^*) \Phi(Z) + \sigma_n(x) \phi(Z)$, where $Z = \frac{\mu_n(x) - f^*}{\sigma_n(x)}$

**UCB:** $\alpha_{UCB}(x) = \mu_n(x) + \sqrt{\beta} \cdot \sigma_n(x)$

In [ ]:
def probability_of_improvement(mu, var, f_best, xi=0.01):
    sigma = np.sqrt(var)
    Z = (mu - f_best - xi) / (sigma + 1e-10)
    return norm.cdf(Z)


def expected_improvement(mu, var, f_best, xi=0.01):
    sigma = np.sqrt(var)
    Z = (mu - f_best - xi) / (sigma + 1e-10)
    ei = (mu - f_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    ei[sigma < 1e-10] = 0.0
    return ei


def upper_confidence_bound(mu, var, beta=2.0):
    return mu + np.sqrt(beta) * np.sqrt(var)

## 3. Visualising All Three on the Same GP Posterior

Same data, same GP — three different strategies for picking the next point.

In [ ]:
true_fn = lambda x: np.sin(x) * np.cos(0.5 * x) + 0.3 * np.sin(3 * x)

X_test = np.linspace(-5, 5, 500).reshape(-1, 1)
X_train = np.array([-3.0, -0.5, 2.5]).reshape(-1, 1)
y_train = true_fn(X_train.ravel())

mu, var = gp_posterior(X_train, y_train, X_test, l=1.0, sf=1.0, noise=1e-4)
std = np.sqrt(var)
f_best = np.max(y_train)

pi_vals = probability_of_improvement(mu, var, f_best)
ei_vals = expected_improvement(mu, var, f_best)
ucb_vals = upper_confidence_bound(mu, var, beta=2.0)

acq_fns = [
    ('PI — Probability of Improvement', pi_vals, '#d4a017', 'exploits mean'),
    ('EI — Expected Improvement', ei_vals, '#2ca02c', 'balances mean + σ'),
    ('UCB — Upper Confidence Bound', ucb_vals, '#7b2d8e', 'high mean + σ'),
]

fig, axes = plt.subplots(2, 3, figsize=(18, 9), gridspec_kw={'height_ratios': [2, 1]})

for col, (name, acq, color, note) in enumerate(acq_fns):
    ax_top = axes[0, col]
    ax_bot = axes[1, col]

    ax_top.fill_between(X_test.ravel(), mu - 2*std, mu + 2*std, alpha=0.15, color='steelblue')
    ax_top.plot(X_test, mu, color='steelblue', linewidth=2, label='μₙ(x)')
    ax_top.plot(X_test, true_fn(X_test.ravel()), 'k--', alpha=0.3, linewidth=1, label='true f(x)')
    ax_top.scatter(X_train, y_train, c='black', s=60, zorder=5)
    ax_top.axhline(f_best, color='#d62728', linestyle='--', alpha=0.5, label='f*')

    x_next_idx = np.argmax(acq)
    x_next = X_test[x_next_idx, 0]
    ax_top.axvline(x_next, color=color, linestyle=':', linewidth=2, alpha=0.8)
    ax_top.set_title(name, fontsize=12, fontweight='bold')
    ax_top.legend(fontsize=8)

    acq_norm = (acq - acq.min()) / (acq.max() - acq.min() + 1e-10)
    ax_bot.fill_between(X_test.ravel(), 0, acq_norm, alpha=0.3, color=color)
    ax_bot.plot(X_test, acq_norm, color=color, linewidth=2)
    ax_bot.axvline(x_next, color=color, linestyle=':', linewidth=2, alpha=0.8)
    ax_bot.scatter([x_next], [1.0], color=color, s=80, zorder=5, marker='v')
    ax_bot.set_xlabel('x')
    ax_bot.set_ylabel('α(x) normalised')
    ax_bot.text(x_next + 0.2, 0.85, f'x_next = {x_next:.2f}\n({note})', fontsize=9, color=color)

fig.suptitle('Same GP Posterior, Three Acquisition Functions → Three Different Next Points', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## 4. Full Bayesian Optimisation Loop

We run BO on a multi-modal test function using EI as the acquisition function, starting from 2 random initial points.

In [ ]:
def bayesian_optimisation(f, bounds, n_init=2, n_iter=15, acq='ei', l=1.0, sf=1.0, beta=2.0):
    X_grid = np.linspace(bounds[0], bounds[1], 1000).reshape(-1, 1)

    X_obs = np.random.uniform(bounds[0], bounds[1], size=(n_init, 1))
    y_obs = f(X_obs.ravel())

    history = [{'X': X_obs.copy(), 'y': y_obs.copy(), 'best': np.max(y_obs)}]

    for i in range(n_iter):
        mu, var = gp_posterior(X_obs, y_obs, X_grid, l=l, sf=sf, noise=1e-4)
        f_best = np.max(y_obs)

        if acq == 'ei':
            acq_vals = expected_improvement(mu, var, f_best)
        elif acq == 'pi':
            acq_vals = probability_of_improvement(mu, var, f_best)
        elif acq == 'ucb':
            acq_vals = upper_confidence_bound(mu, var, beta=beta)

        x_next = X_grid[np.argmax(acq_vals)].reshape(1, 1)
        y_next = f(x_next.ravel())

        X_obs = np.vstack([X_obs, x_next])
        y_obs = np.append(y_obs, y_next)

        history.append({'X': X_obs.copy(), 'y': y_obs.copy(), 'best': np.max(y_obs)})

    return X_obs, y_obs, history

In [ ]:
test_fn = lambda x: -(x - 2)**2 * np.sin(3*x) + 2 * np.exp(-0.5 * (x + 1)**2)

bounds = (-5, 5)
X_obs, y_obs, history = bayesian_optimisation(test_fn, bounds, n_init=2, n_iter=15, acq='ei', l=1.0)

X_grid = np.linspace(bounds[0], bounds[1], 1000).reshape(-1, 1)
y_true = test_fn(X_grid.ravel())
x_true_best = X_grid[np.argmax(y_true), 0]
y_true_best = np.max(y_true)

print(f'True optimum:  x* = {x_true_best:.3f},  f(x*) = {y_true_best:.3f}')
print(f'BO found:      x̂* = {X_obs[np.argmax(y_obs), 0]:.3f},  f(x̂*) = {np.max(y_obs):.3f}')
print(f'Gap (simple regret): {y_true_best - np.max(y_obs):.4f}')

## 5. Visualising the BO Loop — Step by Step

In [ ]:
steps_to_show = [0, 2, 5, 10, 15]
fig, axes = plt.subplots(1, len(steps_to_show), figsize=(20, 4), sharey=True)

for ax, step in zip(axes, steps_to_show):
    h = history[step]
    X_t, y_t = h['X'], h['y']

    if len(X_t) >= 2:
        mu, var = gp_posterior(X_t, y_t, X_grid, l=1.0, sf=1.0, noise=1e-4)
        std = np.sqrt(var)
        ax.fill_between(X_grid.ravel(), mu - 2*std, mu + 2*std, alpha=0.15, color='steelblue')
        ax.plot(X_grid, mu, color='steelblue', linewidth=2)

    ax.plot(X_grid, y_true, 'k--', alpha=0.3, linewidth=1)
    ax.scatter(X_t, y_t, c='black', s=40, zorder=5)

    best_idx = np.argmax(y_t)
    ax.scatter(X_t[best_idx], y_t[best_idx], c='#d62728', s=100, zorder=6, marker='*', label=f'best = {y_t[best_idx]:.2f}')

    n_total = len(X_t)
    ax.set_title(f'n = {n_total} points', fontsize=11, fontweight='bold')
    ax.set_xlabel('x')
    ax.legend(fontsize=8, loc='lower left')

axes[0].set_ylabel('f(x)')
fig.suptitle('Bayesian Optimisation with EI — Progressive Convergence', fontsize=15, y=1.05)
plt.tight_layout()
plt.show()

## 6. BO vs Random Search — Convergence Comparison

In [ ]:
n_runs = 20
n_total = 17

bo_bests = []
rand_bests = []

for run in range(n_runs):
    np.random.seed(run)
    _, _, hist = bayesian_optimisation(test_fn, bounds, n_init=2, n_iter=15, acq='ei', l=1.0)
    bo_bests.append([h['best'] for h in hist])

    np.random.seed(run)
    X_rand = np.random.uniform(bounds[0], bounds[1], size=n_total)
    y_rand = test_fn(X_rand)
    rand_bests.append([np.max(y_rand[:i+1]) for i in range(n_total)])

bo_bests = np.array(bo_bests)
rand_bests = np.array(rand_bests)

fig, ax = plt.subplots(figsize=(10, 6))

iters = np.arange(n_total)
ax.fill_between(iters, np.percentile(bo_bests, 25, axis=0), np.percentile(bo_bests, 75, axis=0),
                alpha=0.2, color='steelblue')
ax.plot(iters, np.median(bo_bests, axis=0), linewidth=2.5, color='steelblue', label='BO (EI) — median')

ax.fill_between(iters, np.percentile(rand_bests, 25, axis=0), np.percentile(rand_bests, 75, axis=0),
                alpha=0.2, color='#d62728')
ax.plot(iters, np.median(rand_bests, axis=0), linewidth=2.5, color='#d62728', label='Random search — median')

ax.axhline(y_true_best, color='black', linestyle='--', alpha=0.5, label=f'True optimum = {y_true_best:.2f}')

ax.set_xlabel('Number of evaluations')
ax.set_ylabel('Best f(x) found')
ax.set_title('BO vs Random Search — 20 runs, shaded IQR')
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

## 7. Exploration–Exploitation: The UCB β Spectrum

β controls the width of the confidence bound:
- β → 0: pure exploitation (follow the mean)
- β → ∞: pure exploration (follow uncertainty)

In [ ]:
betas = [0.01, 0.5, 2.0, 5.0, 20.0]
beta_labels = ['β=0.01\n(exploit)', 'β=0.5\n(mild)', 'β=2.0\n(balanced)', 'β=5.0\n(explore)', 'β=20.0\n(max explore)']
colors = ['#d62728', '#d4a017', '#2ca02c', '#1f77b4', '#7b2d8e']

mu, var = gp_posterior(X_train, y_train, X_test, l=1.0, sf=1.0, noise=1e-4)
std = np.sqrt(var)

fig, axes = plt.subplots(2, 5, figsize=(22, 7), gridspec_kw={'height_ratios': [2, 1]})

for col, (beta, label, color) in enumerate(zip(betas, beta_labels, colors)):
    ax_top = axes[0, col]
    ax_bot = axes[1, col]

    ax_top.fill_between(X_test.ravel(), mu - 2*std, mu + 2*std, alpha=0.12, color='steelblue')
    ax_top.plot(X_test, mu, color='steelblue', linewidth=1.5)
    ax_top.scatter(X_train, y_train, c='black', s=40, zorder=5)

    ucb = upper_confidence_bound(mu, var, beta=beta)
    ax_top.plot(X_test, ucb, color=color, linewidth=1.5, linestyle='--', alpha=0.7, label=f'μ+√β·σ')

    x_next_idx = np.argmax(ucb)
    x_next = X_test[x_next_idx, 0]
    ax_top.axvline(x_next, color=color, linestyle=':', linewidth=2, alpha=0.7)
    ax_top.set_title(label, fontsize=10, fontweight='bold')
    ax_top.legend(fontsize=7)

    ucb_norm = (ucb - ucb.min()) / (ucb.max() - ucb.min() + 1e-10)
    ax_bot.fill_between(X_test.ravel(), 0, ucb_norm, alpha=0.3, color=color)
    ax_bot.plot(X_test, ucb_norm, color=color, linewidth=1.5)
    ax_bot.axvline(x_next, color=color, linestyle=':', linewidth=2, alpha=0.7)
    ax_bot.set_xlabel('x')

fig.suptitle('UCB Exploration–Exploitation Spectrum — Same GP, Different β', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

## 8. Cumulative Regret — Convergence in Practice

Simple regret $r_T = f(x^*) - \max_{t \leq T} f(x_t)$ and cumulative regret $R_T = \sum_{t=1}^T [f(x^*) - f(x_t)]$.

In [ ]:
np.random.seed(42)
X_bo, y_bo, hist_bo = bayesian_optimisation(test_fn, bounds, n_init=2, n_iter=25, acq='ei', l=1.0)

simple_regret = [y_true_best - h['best'] for h in hist_bo]
cumulative_regret = np.cumsum([y_true_best - y for y in y_bo])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(simple_regret, 'o-', color='steelblue', linewidth=2, markersize=5)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Simple Regret r_T')
ax1.set_title('Simple Regret — Gap to True Optimum')
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)

ax2.plot(cumulative_regret, 'o-', color='#d62728', linewidth=2, markersize=5)
ax2.set_xlabel('Evaluation number t')
ax2.set_ylabel('Cumulative Regret R_T')
ax2.set_title('Cumulative Regret — Sublinear Growth = No-Regret')

t = np.arange(1, len(cumulative_regret) + 1)
ax2.plot(t, cumulative_regret[-1] * np.sqrt(t) / np.sqrt(t[-1]), '--', color='gray', alpha=0.5, label='O(√T) reference')
ax2.legend()

plt.tight_layout()
plt.show()

---

**Previous:** [Notebook 1 — GP Surrogate](./01_gp_surrogate.ipynb) · [Notebook 2 — Kernel Design](./02_kernels.ipynb)

**Back to article:** [Bayesian Optimisation — Mathematical Deep Dive](https://omkarray.com/bayesian-optimization.html)